In [1]:
import plotly.express as px
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def extract_xlsx(xlsx_name, ws_name):
  xlsx = pd.ExcelFile(xlsx_name)
  return pd.read_excel(xlsx, ws_name)

In [3]:
xlsx_file = extract_xlsx('Data/OverdoseDeathWA.xlsx', 'By Location and Date')
xlsx_file

,Year,Time Aggregation,Location,Geography,Drug Category,Death Count,Population,Crude Death Rate,Crude Lower CI,Crude Upper CI,Adjusted Death Rate,Adjusted Lower CI,Adjusted Upper CI,Date-Time Stamp
0,2023.3,Quarterly counts,Adams County,County,Cocaine,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-25 08:33:43.039
1,2023.3,Quarterly counts,Asotin County,County,Cocaine,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-25 08:33:43.039
2,2023.3,Quarterly counts,Benton-Franklin,County,Cocaine,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-25 08:33:43.039
3,2023.3,Quarterly counts,Benton County,County,Cocaine,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-25 08:33:43.039
4,2023.3,Quarterly counts,Chelan-Douglas,County,Cocaine,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-25 08:33:43.039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17383,2021.0,5 year rolling counts,Statewide,Statewide,Psychostimulants,3273,37596680.0,8.71,8.41,9.01,8.53,8.23,8.83,2023-07-25 08:33:43.039
17384,2022.0,5 year rolling counts,Statewide,Statewide,Psychostimulants,4242,37942580.0,11.18,10.85,11.52,10.94,10.6,11.28,2023-07-25 08:33:43.039
17385,2020.0,5 year rolling counts,Statewide,Statewide,Synthetic Opioids,1466,37124180.0,3.95,3.75,4.16,4.03,3.83,4.25,2023-07-25 08:33:43.039
17386,2021.0,5 year rolling counts,Statewide,Statewide,Synthetic Opioids,2589,37596680.0,6.89,6.63,7.16,7.04,6.77,7.32,2023-07-25 08:33:43.039


In [4]:
def merge_geo(shp_file_name, xlsx_file=None, csv_file_name=None):
  shp_file = gpd.read_file(shp_file_name)
  if  xlsx_file is None:
    csv_file = pd.read_csv(csv_file_name)
    csv_file = csv_file[(csv_file['State Name'] != 'Alaska') & (csv_file['State Name'] != 'Hawaii')]
    merged_data = shp_file.merge(csv_file, left_on='NAMELSAD', right_on='State Name', how='outer')
  else:
    merged_data = shp_file.merge(xlsx_file, left_on='NAMELSAD', right_on='Location')
  return merged_data

In [6]:
wa_geo_data = merge_geo('Data/geodata/cb_2022_us_county_500k.shp', xlsx_file)
wa_geo_data

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,...,Drug Category,Death Count,Population,Crude Death Rate,Crude Lower CI,Crude Upper CI,Adjusted Death Rate,Adjusted Lower CI,Adjusted Upper CI,Date-Time Stamp
0,12,037,00306911,0500000US12037,12037,Franklin,Franklin County,FL,Florida,06,...,Cocaine,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-25 08:33:43.039
1,12,037,00306911,0500000US12037,12037,Franklin,Franklin County,FL,Florida,06,...,Heroin,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-25 08:33:43.039
2,12,037,00306911,0500000US12037,12037,Franklin,Franklin County,FL,Florida,06,...,Psychostimulants,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-25 08:33:43.039
3,12,037,00306911,0500000US12037,12037,Franklin,Franklin County,FL,Florida,06,...,Any Drug,*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-25 08:33:43.039
4,12,037,00306911,0500000US12037,12037,Franklin,Franklin County,FL,Florida,06,...,Any Drug,*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-25 08:33:43.039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63107,53,007,01531932,0500000US53007,53007,Chelan,Chelan County,WA,Washington,06,...,Psychostimulants,23,392370.0,5.86,3.91,8.8,6.56,4.05,10.14,2023-07-25 08:33:43.039
63108,53,007,01531932,0500000US53007,53007,Chelan,Chelan County,WA,Washington,06,...,Psychostimulants,29,395200.0,7.34,5.11,10.54,7.92,5.17,11.7,2023-07-25 08:33:43.039
63109,53,007,01531932,0500000US53007,53007,Chelan,Chelan County,WA,Washington,06,...,Synthetic Opioids,11,388620.0,2.83,1.58,5.07,3.25,1.59,6.02,2023-07-25 08:33:43.039
63110,53,007,01531932,0500000US53007,53007,Chelan,Chelan County,WA,Washington,06,...,Synthetic Opioids,13,392370.0,3.31,1.94,5.67,4.21,2.22,7.31,2023-07-25 08:33:43.039


In [9]:
def drug_overdose_change(data, start=2015.0,
                         end=2023.0) -> None:

  data = data[data['STATE_NAME'] == 'Washington']
  drug = data['Drug Category'] == 'Any Drug'
  county = data['Geography'] == 'County'
  year = (data['Year'] >= start) & (data['Year'] <= end)
  time = (data['Time Aggregation'] == '1 year rolling counts')
  remove_star = data['Death Count'] != '*'
  county_data = data[drug & county & time & year & remove_star]
  county_data['Death Count'] = county_data['Death Count'].astype('int')
  county_data = county_data.dissolve(by='Year', aggfunc='sum').reset_index()
  fig = px.line(county_data, x="Year", y="Death Count",
                title=f"Drug Overdose Deaths in WA between {int(start)} and {int(end)}", markers=True)
  fig.update_layout(title_x=0.5, title_y=0.95,  font=dict(size=20))
  fig.update_traces(line=dict(width=4), marker=dict(size=10))

  fig.show()

In [ ]:
ans1 = drug_overdose_change(wa_geo_data)
ans1